In [10]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

/kaggle/input/entovinlp/tst2012.vi
/kaggle/input/entovinlp/train.vi
/kaggle/input/entovinlp/tst2013.en
/kaggle/input/entovinlp/tst2013.vi
/kaggle/input/entovinlp/tst2012.en
/kaggle/input/entovinlp/train.en


In [11]:
# --- CELL 1: SETUP ---
import os
import random
import torch
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim
from torch.utils.data import Dataset, DataLoader
import tqdm
import warnings

# Cài đặt thư viện tokenizers nếu chưa có
try:
    import tokenizers
except ImportError:
    os.system('pip install tokenizers')
    import tokenizers

from tokenizers import Tokenizer, models, pre_tokenizers, decoders, trainers

# Cấu hình thiết bị và Random Seed
warnings.filterwarnings("ignore")
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
print(f"🔹 Đang sử dụng thiết bị: {device}")

SEED = 42
random.seed(SEED)
torch.manual_seed(SEED)
if torch.cuda.is_available():
    torch.cuda.manual_seed_all(SEED)

🔹 Đang sử dụng thiết bị: cuda


In [12]:
import re
import html
import unicodedata
"""
    Hàm làm sạch cơ bản, dùng CHUNG cho cả train.vi, train.en và câu input khi dịch.
    Mục tiêu:
      - Bỏ khoảng trắng thừa, ký tự vô hình
      - Giải mã HTML (&quot; -> ")
      - Chuẩn hóa Unicode NFC (dấu tiếng Việt)
      - Ép về chữ thường (rất quan trọng vì Field(lower=True))
      - Chuẩn hóa dấu câu: tách , . ! ? ; : () " ' ra khỏi từ
    """
def clean_text(text: str) -> str:
    if not isinstance(text, str):
        return ""

    text = text.strip()
    text = unicodedata.normalize("NFC", text)
    text = text.lower()
    text = re.sub(r"<[^>]+>", " ", text)
    if not text:
        return ""

    # 1. Giải mã html
    text = html.unescape(text)

    # 2. Expand contraction (như anh đã viết)
    split_patterns = [
        # BE (am/is/are)
        (r"\bi\s*m\b", "i am"),
        (r"\byou\s*re\b", "you are"),
        (r"\bwe\s*re\b", "we are"),
        (r"\bthey\s*re\b", "they are"),
        (r"\bhe\s*s\b", "he is"),
        (r"\bshe\s*s\b", "she is"),
        (r"\bit\s*s\b", "it is"),
        (r"\bthat\s*s\b", "that is"),
        (r"\bthere\s*s\b", "there is"),
        (r"\bhere\s*s\b", "here is"),
    
        # HAVE
        (r"\bi\s*ve\b", "i have"),
        (r"\byou\s*ve\b", "you have"),
        (r"\bwe\s*ve\b", "we have"),
        (r"\bthey\s*ve\b", "they have"),
    
        # WILL
        (r"\bi\s*ll\b", "i will"),
        (r"\byou\s*ll\b", "you will"),
        (r"\bwe\s*ll\b", "we will"),
        (r"\bthey\s*ll\b", "they will"),
        (r"\bhe\s*ll\b", "he will"),
        (r"\bshe\s*ll\b", "she will"),
        (r"\bit\s*ll\b", "it will"),
    
        # WOULD
        (r"\bi\s*d\b", "i would"),
        (r"\byou\s*d\b", "you would"),
        (r"\bwe\s*d\b", "we would"),
        (r"\bthey\s*d\b", "they would"),
        (r"\bhe\s*d\b", "he would"),
        (r"\bshe\s*d\b", "she would"),
        (r"\bit\s*d\b", "it would"),
        (r"\bthat\s*d\b", "that would"),
    
        # NOT (full)
        (r"\bdon\s*t\b", "do not"),
        (r"\bdoesn\s*t\b", "does not"),
        (r"\bdidn\s*t\b", "did not"),
        (r"\bcan\s*t\b", "can not"),
        (r"\bcouldn\s*t\b", "could not"),
        (r"\bshouldn\s*t\b", "should not"),
        (r"\bwouldn\s*t\b", "would not"),
        (r"\bisn\s*t\b", "is not"),
        (r"\baren\s*t\b", "are not"),
        (r"\bwasn\s*t\b", "was not"),
        (r"\bweren\s*t\b", "were not"),
        (r"\bwon\s*t\b", "will not"),
        (r"\bmustn\s*t\b", "must not"),
        (r"\bneedn\s*t\b", "need not"),
        (r"\bmightn\s*t\b", "might not"),
    
        # OTHER
        (r"\blet\s*s\b", "let us"),
    ]

    for pat, repl in split_patterns:
        text = re.sub(pat, repl, text)

    full_patterns = [
        # BE
        (r"\bi m\b", "i am"),
        (r"\byou re\b", "you are"),
        (r"\bwe re\b", "we are"),
        (r"\bthey re\b", "they are"),
        (r"\bhe s\b", "he is"),
        (r"\bshe s\b", "she is"),
        (r"\bit s\b", "it is"),
        (r"\bthat s\b", "that is"),
        (r"\bhere s\b", "here is"),
        (r"\bthere s\b", "there is"),
    
        # HAVE
        (r"\bi ve\b", "i have"),
        (r"\byou ve\b", "you have"),
        (r"\bwe ve\b", "we have"),
        (r"\bthey ve\b", "they have"),

        # WILL
        (r"\bi ll\b", "I will"),
        (r"\byou ll\b", "You will"),
        (r"\bwe ll\b", "We will"),
        (r"\bthey ll\b", "They will"),
        (r"\bhe ll\b", "He will"),
        (r"\bshe ll\b", "She will"),
        (r"\bit ll\b", "It will"),
    
        # WOULD
        (r"\bi d\b", "I would"),
        (r"\byou d\b", "You would"),
        (r"\bwe d\b", "We would"),
        (r"\bthey d\b", "They would"),
        (r"\bhe d\b", "He would"),
        (r"\bshe d\b", "She would"),
        (r"\bit d\b", "It would"),
        (r"\bthat d\b", "That would"),
    
        # NOT
        (r"\bdon t\b", "do not"),
        (r"\bdoesn t\b", "does not"),
        (r"\bdidn t\b", "did not"),
        (r"\bcan t\b", "can not"),
        (r"\bcouldn t\b", "could not"),
        (r"\bshouldn t\b", "should not"),
        (r"\bwouldn t\b", "would not"),
        (r"\bisn t\b", "is not"),
        (r"\baren t\b", "are not"),
        (r"\bwasn t\b", "was not"),
        (r"\bweren t\b", "were not"),
        (r"\bwon t\b", "will not"),
        (r"\bmustn t\b", "must not"),
        (r"\bneedn t\b", "need not"),
        (r"\bmightn t\b", "might not"),
    
        # OTHER
        (r"\blet s\b", "let us"),
    ]

    for pat, repl in full_patterns:
        text = re.sub(pat, repl, text)

    # 3. Bỏ ký tự vô hình + bỏ --, ---, ...
    text = text.replace("\u200b", "")
    text = re.sub(r"-{2,}", " ", text)

    # 4. Gộp nhiều space liên tiếp
    text = re.sub(r"\s+", " ", text)
    
    text = re.sub(r"\s+", " ", text).strip()
    return text

In [13]:
# --- CELL 2: DATA READING FUNCTION ---
def read_parallel_files(src_filename, tgt_filename):
    """Đọc cặp file song ngữ, trả về list các tuple (câu_nguồn, câu_đích)"""
    # Kiểm tra đường dẫn (hỗ trợ cả thư mục hiện tại và thư mục input của Kaggle)
    possible_paths = ["./", "/kaggle/input/", "/kaggle/working/"]
    
    src_path, tgt_path = None, None
    for p in possible_paths:
        if os.path.exists(os.path.join(p, src_filename)):
            src_path = os.path.join(p, src_filename)
        if os.path.exists(os.path.join(p, tgt_filename)):
            tgt_path = os.path.join(p, tgt_filename)
            
    if not src_path or not tgt_path:
        print(f"⚠️ Không tìm thấy file {src_filename} hoặc {tgt_filename}. Bỏ qua.")
        return []

    print(f"📖 Đang đọc: {src_path} và {tgt_path}")
    with open(src_path, 'r', encoding='utf-8') as f_src, \
         open(tgt_path, 'r', encoding='utf-8') as f_tgt:
        src_lines = [clean_text(line.strip()) for line in f_src.read().splitlines()]
        tgt_lines = [clean_text(line.strip()) for line in f_tgt.read().splitlines()]
    
    # Lọc bỏ các cặp câu rỗng hoặc lệch dòng
    pairs = []
    min_len = min(len(src_lines), len(tgt_lines))
    for i in range(min_len):
        if src_lines[i] and tgt_lines[i]:
            pairs.append((src_lines[i], tgt_lines[i]))
            
    return pairs

In [14]:
# --- CELL 3: LOAD DATA ---
print("\n--- ĐANG TẢI DỮ LIỆU ---")
# Đảm bảo tên file khớp với file bạn upload
train_pairs = read_parallel_files("/kaggle/input/entovinlp/train.en", "/kaggle/input/entovinlp/train.vi")
val_pairs = read_parallel_files("/kaggle/input/entovinlp/tst2012.en", "/kaggle/input/entovinlp/tst2012.vi")
test_pairs = read_parallel_files("/kaggle/input/entovinlp/tst2013.en", "/kaggle/input/entovinlp/tst2013.vi")

print(f"✅ Train size: {len(train_pairs)}")
print(f"✅ Val size: {len(val_pairs)}")
print(f"✅ Test size: {len(test_pairs)}")

if len(train_pairs) > 0:
    print(f"🔎 Ví dụ mẫu: {train_pairs[0:5]}")


--- ĐANG TẢI DỮ LIỆU ---
📖 Đang đọc: /kaggle/input/entovinlp/train.en và /kaggle/input/entovinlp/train.vi
📖 Đang đọc: /kaggle/input/entovinlp/tst2012.en và /kaggle/input/entovinlp/tst2012.vi
📖 Đang đọc: /kaggle/input/entovinlp/tst2013.en và /kaggle/input/entovinlp/tst2013.vi
✅ Train size: 133166
✅ Val size: 1553
✅ Test size: 1268
🔎 Ví dụ mẫu: [('rachel pike : the science behind a climate headline', 'khoa học đằng sau một tiêu đề về khí hậu'), ('in 4 minutes , atmospheric chemist rachel pike provides a glimpse of the massive scientific effort behind the bold headlines on climate change , with her team one of thousands who contributed taking a risky flight over the rainforest in pursuit of data on a key molecule .', 'trong 4 phút , chuyên gia hoá học khí quyển rachel pike giới thiệu sơ lược về những nỗ lực khoa học miệt mài đằng sau những tiêu đề táo bạo về biến đổi khí hậu , cùng với đoàn nghiên cứu của mình hàng ngàn người đã cống hiến cho dự án này một chuyến bay mạo hiểm qua rừng gi

In [15]:
# --- CELL 4: TRAIN TOKENIZERS ---
print("\n--- HUẤN LUYỆN TOKENIZER ---")

def train_bpe_tokenizer(texts, vocab_size=8000):
    tokenizer = Tokenizer(models.BPE())
    tokenizer.pre_tokenizer = pre_tokenizers.ByteLevel(add_prefix_space=True)
    tokenizer.decoder = decoders.ByteLevel()
    
    trainer = trainers.BpeTrainer(
        vocab_size=vocab_size,
        special_tokens=["[PAD]", "[START]", "[END]", "[UNK]"],
        show_progress=False
    )
    tokenizer.train_from_iterator(texts, trainer=trainer)
    return tokenizer

# Gộp text để train tokenizer
all_src_text = [p[0] for p in train_pairs + val_pairs]
all_tgt_text = [p[1] for p in train_pairs + val_pairs]

if not all_src_text: # Dummy data nếu chưa load được file
    all_src_text = ["Hello world"]
    all_tgt_text = ["Xin chào"]

en_tokenizer = train_bpe_tokenizer(all_src_text, vocab_size=10000)
vi_tokenizer = train_bpe_tokenizer(all_tgt_text, vocab_size=10000)

# Lấy ID các token đặc biệt
PAD_ID = en_tokenizer.token_to_id("[PAD]")
START_ID = vi_tokenizer.token_to_id("[START]")
END_ID = vi_tokenizer.token_to_id("[END]")

print("✅ Tokenizer đã sẵn sàng.")


--- HUẤN LUYỆN TOKENIZER ---
✅ Tokenizer đã sẵn sàng.


In [16]:
# --- CELL 5: DATASET CLASS ---
class EnViDataset(Dataset):
    def __init__(self, pairs):
        self.pairs = pairs

    def __len__(self):
        return len(self.pairs)

    def __getitem__(self, idx):
        return self.pairs[idx]

def collate_fn(batch):
    en_batch, vi_batch = zip(*batch)
    
    # Encode tiếng Anh (Source)
    en_enc = en_tokenizer.encode_batch(list(en_batch))
    en_ids = [e.ids for e in en_enc]
    
    # Encode tiếng Việt (Target) - Thêm START và END thủ công
    vi_ids = []
    for text in vi_batch:
        ids = vi_tokenizer.encode(text).ids
        vi_ids.append([START_ID] + ids + [END_ID])
    
    # Padding
    max_len_en = max([len(x) for x in en_ids])
    max_len_vi = max([len(x) for x in vi_ids])
    
    padded_en = [x + [PAD_ID] * (max_len_en - len(x)) for x in en_ids]
    padded_vi = [x + [PAD_ID] * (max_len_vi - len(x)) for x in vi_ids]
    
    return torch.tensor(padded_en), torch.tensor(padded_vi)

In [17]:
# --- CELL 6: DATALOADERS ---
BATCH_SIZE = 32

train_loader = DataLoader(EnViDataset(train_pairs), batch_size=BATCH_SIZE, shuffle=True, collate_fn=collate_fn)
val_loader = DataLoader(EnViDataset(val_pairs), batch_size=BATCH_SIZE, shuffle=False, collate_fn=collate_fn)

print(f"DataLoaders created. Batch size: {BATCH_SIZE}")

DataLoaders created. Batch size: 32


In [18]:
# --- CELL 7: MODEL COMPONENTS ---
# --- Rotary Positional Embeddings ---
def rotate_half(x):
    x1, x2 = x.chunk(2, dim=-1)
    return torch.cat((-x2, x1), dim=-1)

def apply_rotary_pos_emb(x, cos, sin):
    return (x * cos) + (rotate_half(x) * sin)

class RotaryPositionalEncoding(nn.Module):
    def __init__(self, head_dim, max_seq_len=2048):
        super().__init__()
        inv_freq = 1.0 / (10000 ** (torch.arange(0, head_dim, 2).float() / head_dim))
        t = torch.arange(max_seq_len).float()
        freqs = torch.outer(t, inv_freq)
        emb = torch.cat((freqs, freqs), dim=-1)
        self.register_buffer("cos", emb.cos()[None, :, None, :])
        self.register_buffer("sin", emb.sin()[None, :, None, :])

    def forward(self, x, seq_len):
        return self.cos[:, :seq_len, :, :], self.sin[:, :seq_len, :, :]

# --- Feed Forward (SwiGLU) ---
class SwiGLU(nn.Module):
    def __init__(self, hidden_dim, intermediate_dim):
        super().__init__()
        self.w1 = nn.Linear(hidden_dim, intermediate_dim)
        self.w2 = nn.Linear(hidden_dim, intermediate_dim)
        self.w3 = nn.Linear(intermediate_dim, hidden_dim)

    def forward(self, x):
        return self.w3(F.silu(self.w1(x)) * self.w2(x))

# --- GQA Attention ---
class GQA(nn.Module):
    def __init__(self, hidden_dim, num_heads, num_kv_heads, dropout=0.1):
        super().__init__()
        self.num_heads = num_heads
        self.num_kv_heads = num_kv_heads
        self.head_dim = hidden_dim // num_heads
        self.num_groups = num_heads // num_kv_heads
        
        self.q_proj = nn.Linear(hidden_dim, hidden_dim)
        self.k_proj = nn.Linear(hidden_dim, num_kv_heads * self.head_dim)
        self.v_proj = nn.Linear(hidden_dim, num_kv_heads * self.head_dim)
        self.o_proj = nn.Linear(hidden_dim, hidden_dim)
        self.dropout = dropout

    def forward(self, x, enc_out=None, mask=None, rope_cos=None, rope_sin=None):
        batch, seq_len, _ = x.shape
        kv_input = enc_out if enc_out is not None else x
        kv_seq_len = kv_input.shape[1]

        q = self.q_proj(x).view(batch, seq_len, self.num_heads, self.head_dim)
        k = self.k_proj(kv_input).view(batch, kv_seq_len, self.num_kv_heads, self.head_dim)
        v = self.v_proj(kv_input).view(batch, kv_seq_len, self.num_kv_heads, self.head_dim)

        if rope_cos is not None and enc_out is None:
            q = apply_rotary_pos_emb(q, rope_cos, rope_sin)
            k = apply_rotary_pos_emb(k, rope_cos, rope_sin)

        q, k, v = q.transpose(1, 2), k.transpose(1, 2), v.transpose(1, 2)

        if self.num_groups > 1:
            k = k[:, :, None, :, :].expand(batch, self.num_kv_heads, self.num_groups, kv_seq_len, self.head_dim).reshape(batch, self.num_heads, kv_seq_len, self.head_dim)
            v = v[:, :, None, :, :].expand(batch, self.num_kv_heads, self.num_groups, kv_seq_len, self.head_dim).reshape(batch, self.num_heads, kv_seq_len, self.head_dim)

        out = F.scaled_dot_product_attention(q, k, v, attn_mask=mask, dropout_p=self.dropout if self.training else 0.0)
        return self.o_proj(out.transpose(1, 2).reshape(batch, seq_len, -1))

In [19]:
# --- CELL 8: TRANSFORMER MODEL ---
class TransformerBlock(nn.Module):
    def __init__(self, hidden_dim, num_heads, num_kv_heads, dropout=0.1, is_decoder=False):
        super().__init__()
        self.norm1 = nn.RMSNorm(hidden_dim)
        self.attn = GQA(hidden_dim, num_heads, num_kv_heads, dropout)
        self.is_decoder = is_decoder
        if is_decoder:
            self.norm2 = nn.RMSNorm(hidden_dim)
            self.cross_attn = GQA(hidden_dim, num_heads, num_kv_heads, dropout)
        self.norm_ffn = nn.RMSNorm(hidden_dim)
        self.ffn = SwiGLU(hidden_dim, hidden_dim * 4)

    def forward(self, x, enc_out=None, mask=None, cross_mask=None, rope_cos=None, rope_sin=None):
        x = x + self.attn(self.norm1(x), mask=mask, rope_cos=rope_cos, rope_sin=rope_sin)
        if self.is_decoder:
            x = x + self.cross_attn(self.norm2(x), enc_out=enc_out, mask=cross_mask)
        x = x + self.ffn(self.norm_ffn(x))
        return x

class Transformer(nn.Module):
    def __init__(self, src_vocab, tgt_vocab, hidden_dim=256, num_layers=4, num_heads=8, num_kv_heads=4):
        super().__init__()
        self.src_emb = nn.Embedding(src_vocab, hidden_dim)
        self.tgt_emb = nn.Embedding(tgt_vocab, hidden_dim)
        self.rope = RotaryPositionalEncoding(hidden_dim // num_heads)
        self.encoders = nn.ModuleList([TransformerBlock(hidden_dim, num_heads, num_kv_heads) for _ in range(num_layers)])
        self.decoders = nn.ModuleList([TransformerBlock(hidden_dim, num_heads, num_kv_heads, is_decoder=True) for _ in range(num_layers)])
        self.final_norm = nn.RMSNorm(hidden_dim)
        self.fc_out = nn.Linear(hidden_dim, tgt_vocab)

    def forward(self, src, tgt, src_mask, tgt_mask):
        x = self.src_emb(src)
        rope_cos, rope_sin = self.rope(x, x.shape[1])
        for layer in self.encoders:
            x = layer(x, mask=src_mask, rope_cos=rope_cos, rope_sin=rope_sin)
        enc_out = x
        
        x = self.tgt_emb(tgt)
        rope_cos_tgt, rope_sin_tgt = self.rope(x, x.shape[1])
        for layer in self.decoders:
            x = layer(x, enc_out=enc_out, mask=tgt_mask, cross_mask=src_mask, rope_cos=rope_cos_tgt, rope_sin=rope_sin_tgt)
        return self.fc_out(self.final_norm(x))

In [20]:
# --- CELL 9: INIT TRAINING ---
def create_masks(src, tgt):
    src_mask = (src == PAD_ID).unsqueeze(1).unsqueeze(2).float() * -1e9
    batch, seq_len = tgt.shape
    causal = torch.triu(torch.full((seq_len, seq_len), float('-inf'), device=device), diagonal=1)
    tgt_pad = (tgt == PAD_ID).unsqueeze(1).unsqueeze(2).float() * -1e9
    return src_mask, causal + tgt_pad

model = Transformer(
    src_vocab=en_tokenizer.get_vocab_size(),
    tgt_vocab=vi_tokenizer.get_vocab_size(),
    hidden_dim=256, num_layers=4, num_heads=8, num_kv_heads=4
).to(device)

criterion = nn.CrossEntropyLoss(ignore_index=PAD_ID, label_smoothing=0.1)
optimizer = optim.AdamW(model.parameters(), lr=0.0005, weight_decay=0.0001)

print("Model Initialized.")

Model Initialized.


In [21]:
# --- CELL 10: TRAINING LOOP ---
EPOCHS = 10
print("\n--- BẮT ĐẦU HUẤN LUYỆN ---")

for epoch in range(EPOCHS):
    model.train()
    train_loss = 0
    pbar = tqdm.tqdm(train_loader, desc=f"Epoch {epoch+1}/{EPOCHS}")
    
    for src, tgt in pbar:
        src, tgt = src.to(device), tgt.to(device)
        tgt_input, tgt_real = tgt[:, :-1], tgt[:, 1:]
        src_mask, tgt_mask = create_masks(src, tgt_input)
        
        optimizer.zero_grad()
        output = model(src, tgt_input, src_mask, tgt_mask)
        loss = criterion(output.reshape(-1, output.shape[-1]), tgt_real.reshape(-1))
        loss.backward()
        torch.nn.utils.clip_grad_norm_(model.parameters(), 1.0)
        optimizer.step()
        
        train_loss += loss.item()
        pbar.set_postfix(loss=f"{loss.item():.4f}")
        
    # Validation
    model.eval()
    val_loss = 0
    with torch.no_grad():
        for src, tgt in val_loader:
            src, tgt = src.to(device), tgt.to(device)
            tgt_input, tgt_real = tgt[:, :-1], tgt[:, 1:]
            src_mask, tgt_mask = create_masks(src, tgt_input)
            output = model(src, tgt_input, src_mask, tgt_mask)
            val_loss += criterion(output.reshape(-1, output.shape[-1]), tgt_real.reshape(-1)).item()
            
    print(f"Epoch {epoch+1} | Train Loss: {train_loss/len(train_loader):.4f} | Val Loss: {val_loss/len(val_loader):.4f}")

torch.save(model.state_dict(), "transformer_en_vi.pth")


--- BẮT ĐẦU HUẤN LUYỆN ---


Epoch 1/10: 100%|██████████| 4162/4162 [06:47<00:00, 10.22it/s, loss=3.7542]


Epoch 1 | Train Loss: 4.3289 | Val Loss: 3.7808


Epoch 2/10: 100%|██████████| 4162/4162 [06:47<00:00, 10.21it/s, loss=3.3598]


Epoch 2 | Train Loss: 3.4575 | Val Loss: 3.5213


...

Epoch 8 | Train Loss: 2.6762 | Val Loss: 3.3188


Epoch 9/10: 100%|██████████| 4162/4162 [06:47<00:00, 10.21it/s, loss=2.5302]


Epoch 9 | Train Loss: 2.6128 | Val Loss: 3.3295


Epoch 10/10: 100%|██████████| 4162/4162 [06:48<00:00, 10.19it/s, loss=2.7367]


Epoch 10 | Train Loss: 2.5550 | Val Loss: 3.3539


In [22]:
# --- CELL 11: INFERENCE ---
print("\n--- TEST KẾT QUẢ ---")
def translate(sentence, max_len=100):
    model.eval()
    with torch.no_grad():
        src = torch.tensor([en_tokenizer.encode(sentence).ids]).to(device)
        src_mask = (src == PAD_ID).unsqueeze(1).unsqueeze(2).float() * -1e9
        tgt_ids = [START_ID]
        
        for _ in range(max_len):
            tgt = torch.tensor([tgt_ids]).to(device)
            causal = torch.triu(torch.full((tgt.shape[1], tgt.shape[1]), float('-inf'), device=device), diagonal=1)
            out = model(src, tgt, src_mask, causal)
            next_token = out[0, -1, :].argmax().item()
            if next_token == END_ID: break
            tgt_ids.append(next_token)
        return vi_tokenizer.decode(tgt_ids[1:])

for i in range(5):
    if len(test_pairs) > 0:
        idx = random.randint(0, len(test_pairs)-1)
        en_txt, vi_txt = test_pairs[idx]
        print(f"🔹 Input:  {en_txt}\n🔸 Target: {vi_txt}\n🚀 Model:  {translate(en_txt)}\n{'-'*50}")


--- TEST KẾT QUẢ ---
🔹 Input:  and he said that he needed those guns because of the trauma he 'd experienced as a young boy .
🔸 Target: và anh ta nói rằng anh ta cần những cây súng này bởi vì những tổn thương mà anh đã trải qua trong quá khứ khi là một đứa trẻ .
🚀 Model:   và ông ấy nói rằng ông cần những con súng đó vì những chấn thương đó ông đã trải qua khi còn là một cậu bé .
--------------------------------------------------
🔹 Input:  am i south korean or north korean ?
🔸 Target: tôi là người nam triều tiên hay bắc triều tiên ?
🚀 Model:   tôi có phải quay về nam hàn hay bắc hàn hay bắc hàn ?
--------------------------------------------------
🔹 Input:  and so just as the womb entirely envelopes the embryo , which grows within it , the divine matrix of compassion nourishes the entire existence .
🔸 Target: và bởi vì tử cung bao bọc hoàn toàn phôi thai đang phát triển trong lòng nó , ma trận thiêng liêng của tình thương nuôi dưỡng toàn bộ sự sống đó .
🚀 Model:   và như là khi môi trư

In [23]:
# Test thủ công 1 câu riêng
custom_sentence = "I really like this model."
print(f"\n🚀 Custom Test Input: {custom_sentence}")
print(f"✅ Model Translation: {translate(custom_sentence)}")


🚀 Custom Test Input: I really like this model.
✅ Model Translation:  ví dụ như mô hình này .


In [24]:
!pip install sacrebleu

huggingface/tokenizers: The current process just got forked, after parallelism has already been used. Disabling parallelism to avoid deadlocks...
To disable this warning, you can either:
	- Avoid using `tokenizers` before the fork if possible
	- Explicitly set the environment variable TOKENIZERS_PARALLELISM=(true | false)


     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 51.8/51.8 kB 1.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 104.1/104.1 kB 4.3 MB/s eta 0:00:00


In [25]:
import sacrebleu
import random
from tqdm import tqdm # Thanh hiển thị tiến độ

def calculate_bleu(data_pairs, num_samples=100):
    print(f"--- 📊 ĐANG TÍNH ĐIỂM BLEU TRÊN {num_samples} MẪU ---")
    
    # Chọn ngẫu nhiên mẫu để test (hoặc lấy hết nếu num_samples=None)
    if num_samples is not None and num_samples < len(data_pairs):
        samples = random.sample(data_pairs, num_samples)
    else:
        samples = data_pairs

    preds = [] # Các câu máy dịch
    refs = []  # Các câu đáp án chuẩn

    # Bắt đầu dịch
    for en_txt, vi_txt in tqdm(samples):
        # Dịch câu tiếng Anh
        pred_sent = translate(en_txt)
        
        preds.append(pred_sent)
        refs.append(vi_txt) # Sacrebleu nhận list các string cho refs

    # Tính điểm BLEU
    # refs cần được bọc trong list vì 1 câu input có thể có nhiều câu target (ở đây ta có 1)
    bleu = sacrebleu.corpus_bleu(preds, [refs])
    
    return bleu.score

# --- CHẠY TÍNH ĐIỂM ---
# Bạn có thể tăng số lượng mẫu lên len(test_pairs) để chính xác hơn (sẽ chạy lâu hơn)
score = calculate_bleu(test_pairs, num_samples=100)

print(f"\n🌟 ĐIỂM BLEU CỦA MODEL: {score:.2f}")

--- 📊 ĐANG TÍNH ĐIỂM BLEU TRÊN 100 MẪU ---


100%|██████████| 100/100 [00:28<00:00,  3.55it/s]


🌟 ĐIỂM BLEU CỦA MODEL: 28.83
